In [ ]:
# default_exp models.RNNPlus

# RNNPlus

> These are RNN, LSTM and GRU PyTorch implementations created by Ignacio Oguiza - timeseriesAI@gmail.com based on:

In [ ]:
#export
from tsai.imports import *
from tsai.utils import *
from tsai.data.core import *
from tsai.models.layers import *

In [ ]:
#export
class _RNN_Backbone(Module):
    def __init__(self, cell, c_in, c_out, seq_len=None, hidden_size=100, n_layers=1, act=None, act_kwargs={}, bias=True, rnn_dropout=0, bidirectional=False):
        layers = []
        for i in range(len(hidden_size)):
            input_size = c_in if i == 0 else hs * (1 + bidirectional)
            hs = hidden_size[i] 
            layers.append(cell(input_size, hs, num_layers=1, bias=bias, batch_first=True, dropout=rnn_dropout, bidirectional=bidirectional))
            layers.append(LSTMOutput()) # this selects just the output, and discards h_n, and c_n
            if act is not None: layers.append(get_act_fn(act, **act_kwargs))
        self.rnn = nn.Sequential(*layers)
        self.transpose = Transpose(-1, -2, contiguous=True)

    def forward(self, x):
        x = x.transpose(2,1)                     # [batch_size x n_vars x seq_len] --> [batch_size x seq_len x n_vars]
        x = self.rnn(x)                          # [batch_size x seq_len x hidden_size * (1 + bidirectional)]
        x = self.transpose(x)                    # [batch_size x hidden_size * (1 + bidirectional) x seq_len]
        return x

In [ ]:
#export
class _RNNPlus_Base(nn.Sequential):
    def __init__(self, c_in, c_out, seq_len=None, hidden_size=[100], n_layers=1, act=None, act_kwargs={}, bias=True, rnn_dropout=0, bidirectional=False, 
                 fc_dropout=0., last_step=True, bn=False, custom_head=None, y_range=None, **kwargs):

        if not last_step: assert seq_len, 'you need to enter a seq_len to use flatten=True'

        # Backbone
        hidden_size = listify(hidden_size)
        if len(hidden_size) == 1 and n_layers != 1: hidden_size = hidden_size * n_layers
        assert len(hidden_size) == n_layers, "hidden size must be a single item or a list with len == n_layers."
        backbone = _RNN_Backbone(self._cell, c_in, c_out, seq_len=seq_len, hidden_size=hidden_size, n_layers=n_layers, act=act, act_kwargs=act_kwargs, 
                                 bias=bias, rnn_dropout=rnn_dropout,  bidirectional=bidirectional)

        # Head
        self.head_nf = hidden_size * (1 + bidirectional) if isinstance(hidden_size, Integral) else hidden_size[-1] * (1 + bidirectional) 
        if custom_head: 
            if isinstance(custom_head, nn.Module): head = custom_head
            else: head = custom_head(self.head_nf, c_out, seq_len) # custom head must have all required kwargs
        else: head = self.create_head(self.head_nf, c_out, seq_len, last_step=last_step, fc_dropout=fc_dropout, bn=bn, y_range=y_range)
        super().__init__(OrderedDict([('backbone', backbone), ('head', head)]))

    def create_head(self, nf, c_out, seq_len, last_step=True, fc_dropout=0., bn=False, y_range=None):
        if last_step:
            layers = [LastStep()]
        else:
            layers = [Flatten()]
            nf *= seq_len
        layers += [LinBnDrop(nf, c_out, bn=bn, p=fc_dropout)]
        if y_range: layers += [SigmoidRange(*y_range)]
        return nn.Sequential(*layers)


class RNNPlus(_RNNPlus_Base):
    _cell = nn.RNN

class LSTMPlus(_RNNPlus_Base):
    _cell = nn.LSTM

class GRUPlus(_RNNPlus_Base):
    _cell = nn.GRU

In [ ]:
bs = 16
c_in = 3
seq_len = 12
c_out = 2
xb = torch.rand(bs, c_in, seq_len)
test_eq(RNNPlus(c_in, c_out)(xb).shape, [bs, c_out])
test_eq(RNNPlus(c_in, c_out, hidden_size=100, n_layers=2, bias=True, rnn_dropout=0.2, bidirectional=True, fc_dropout=0.5)(xb).shape, [bs, c_out])
test_eq(RNNPlus(c_in, c_out, hidden_size=[100, 50], n_layers=2, bias=True, rnn_dropout=0.2, bidirectional=True, fc_dropout=0.5)(xb).shape, [bs, c_out])
test_eq(RNNPlus(c_in, c_out, hidden_size=[100, 50], n_layers=2, act="selu", bias=True, rnn_dropout=0.2, bidirectional=True, fc_dropout=0.5)(xb).shape, 
        [bs, c_out])
test_eq(LSTMPlus(c_in, c_out, hidden_size=100, n_layers=2, bias=True, rnn_dropout=0.2, bidirectional=True, fc_dropout=0.5)(xb).shape, [bs, c_out])
test_eq(GRUPlus(c_in, c_out, hidden_size=100, n_layers=2, bias=True, rnn_dropout=0.2, bidirectional=True, fc_dropout=0.5)(xb).shape, [bs, c_out])
test_eq(RNNPlus(c_in, c_out, seq_len, flatten=True)(xb).shape, [bs, c_out])
test_eq(RNNPlus(c_in, c_out, seq_len, flatten=True)(xb).shape, [bs, c_out])
test_eq(RNNPlus(c_in, c_out, seq_len, hidden_size=100, n_layers=2, bias=True, rnn_dropout=0.2, bidirectional=True, fc_dropout=0.5, flatten=True)(xb).shape, 
        [bs, c_out])
test_eq(LSTMPlus(c_in, c_out, seq_len, flatten=True)(xb).shape, [bs, c_out])
test_eq(GRUPlus(c_in, c_out, seq_len, last_step=False, flatten=True)(xb).shape, [bs, c_out])

/Users/nacho/opt/anaconda3/envs/py36/lib/python3.6/site-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [ ]:
bs = 16
c_in = 3
seq_len = 12
c_out = 2
xb = torch.rand(bs, c_in, seq_len)
custom_head = partial(create_mlp_head, fc_dropout=0.5)
test_eq(LSTMPlus(c_in, c_out, seq_len, flatten=True, custom_head=custom_head)(xb).shape, [bs, c_out])
custom_head = partial(create_pool_head, concat_pool=True, fc_dropout=0.5)
test_eq(LSTMPlus(c_in, c_out, seq_len, last_step=False, flatten=False, custom_head=custom_head)(xb).shape, [bs, c_out])
custom_head = partial(create_pool_plus_head, fc_dropout=0.5)
test_eq(LSTMPlus(c_in, c_out, seq_len, last_step=False, flatten=False, custom_head=custom_head)(xb).shape, [bs, c_out])
custom_head = partial(create_conv_head)
test_eq(LSTMPlus(c_in, c_out, seq_len, last_step=False, flatten=False, custom_head=custom_head)(xb).shape, [bs, c_out])
test_eq(LSTMPlus(c_in, c_out, seq_len, hidden_size=[100, 50], n_layers=2, act="selu", bias=True, rnn_dropout=0.2, bidirectional=True)(xb).shape, [bs, c_out])

In [ ]:
LSTMPlus(c_in, c_out, seq_len, hidden_size=[100, 50], n_layers=2, act="selu", bias=True, rnn_dropout=0.2, bidirectional=True)

LSTMPlus(
  (backbone): _RNN_Backbone(
    (rnn): Sequential(
      (0): LSTM(3, 100, batch_first=True, dropout=0.2, bidirectional=True)
      (1): LSTMOutput()
      (2): SELU()
      (3): LSTM(200, 50, batch_first=True, dropout=0.2, bidirectional=True)
      (4): LSTMOutput()
      (5): SELU()
    )
    (transpose): Transpose(dims=-1, -2).contiguous()
  )
  (head): Sequential(
    (0): LastStep()
    (1): LinBnDrop(
      (0): Linear(in_features=100, out_features=2, bias=True)
    )
  )
)

In [ ]:
from tsai.data.all import *
from tsai.models.utils import *
dsid = 'NATOPS' 
bs = 16
X, y, splits = get_UCR_data(dsid, return_split=False)
tfms  = [None, [Categorize()]]
dls = get_ts_dls(X, y, tfms=tfms, splits=splits, bs=bs)

In [ ]:
model = build_ts_model(LSTMPlus, dls=dls)
print(model[-1])
learn = Learner(dls, model,  metrics=accuracy)
learn.fit_one_cycle(1, 3e-3)

Sequential(
  (0): LastStep()
  (1): LinBnDrop(
    (0): Linear(in_features=100, out_features=6, bias=True)
  )
)


epoch,train_loss,valid_loss,accuracy,time
0,1.808975,1.792509,0.177778,00:01


In [ ]:
model = LSTMPlus(dls.vars, dls.c, dls.len, last_step=False, flatten=True)
learn = Learner(dls, model,  metrics=accuracy)
learn.fit_one_cycle(1, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.240743,0.694240,0.716667,00:01


In [ ]:
custom_head = partial(create_pool_head, concat_pool=True)
model = LSTMPlus(dls.vars, dls.c, dls.len, last_step=False, flatten=False, custom_head=custom_head)
learn = Learner(dls, model,  metrics=accuracy)
learn.fit_one_cycle(1, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.733672,1.638750,0.505556,00:01


In [ ]:
custom_head = partial(create_pool_plus_head, concat_pool=True)
model = LSTMPlus(dls.vars, dls.c, dls.len, last_step=False, flatten=False, custom_head=custom_head)
learn = Learner(dls, model,  metrics=accuracy)
learn.fit_one_cycle(1, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.127303,1.594975,0.522222,00:01


In [ ]:
m = RNNPlus(c_in, c_out, seq_len, hidden_size=100,n_layers=2,bidirectional=True,rnn_dropout=.5,fc_dropout=.5, flatten=True)
print(m)
print(total_params(m))
m(xb).shape

RNNPlus(
  (backbone): _RNN_Backbone(
    (rnn): Sequential(
      (0): RNN(3, 100, batch_first=True, dropout=0.5, bidirectional=True)
      (1): LSTMOutput()
      (2): RNN(200, 100, batch_first=True, dropout=0.5, bidirectional=True)
      (3): LSTMOutput()
    )
    (transpose): Transpose(dims=-1, -2).contiguous()
  )
  (head): Sequential(
    (0): LastStep()
    (1): LinBnDrop(
      (0): Dropout(p=0.5, inplace=False)
      (1): Linear(in_features=200, out_features=2, bias=True)
    )
  )
)
(81802, True)


/Users/nacho/opt/anaconda3/envs/py36/lib/python3.6/site-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


torch.Size([16, 2])

In [ ]:
m = LSTMPlus(c_in, c_out, seq_len, hidden_size=100,n_layers=2,bidirectional=True,rnn_dropout=.5,fc_dropout=.5, flatten=True)
print(m)
print(total_params(m))
m(xb).shape

LSTMPlus(
  (backbone): _RNN_Backbone(
    (rnn): Sequential(
      (0): LSTM(3, 100, batch_first=True, dropout=0.5, bidirectional=True)
      (1): LSTMOutput()
      (2): LSTM(200, 100, batch_first=True, dropout=0.5, bidirectional=True)
      (3): LSTMOutput()
    )
    (transpose): Transpose(dims=-1, -2).contiguous()
  )
  (head): Sequential(
    (0): LastStep()
    (1): LinBnDrop(
      (0): Dropout(p=0.5, inplace=False)
      (1): Linear(in_features=200, out_features=2, bias=True)
    )
  )
)
(326002, True)


torch.Size([16, 2])

In [ ]:
m = GRUPlus(c_in, c_out, seq_len, hidden_size=100,n_layers=2,bidirectional=True,rnn_dropout=.5,fc_dropout=.5, flatten=True)
print(m)
print(total_params(m))
m(xb).shape

GRUPlus(
  (backbone): _RNN_Backbone(
    (rnn): Sequential(
      (0): GRU(3, 100, batch_first=True, dropout=0.5, bidirectional=True)
      (1): LSTMOutput()
      (2): GRU(200, 100, batch_first=True, dropout=0.5, bidirectional=True)
      (3): LSTMOutput()
    )
    (transpose): Transpose(dims=-1, -2).contiguous()
  )
  (head): Sequential(
    (0): LastStep()
    (1): LinBnDrop(
      (0): Dropout(p=0.5, inplace=False)
      (1): Linear(in_features=200, out_features=2, bias=True)
    )
  )
)
(244602, True)


torch.Size([16, 2])

In [ ]:
#hide
from tsai.imports import create_scripts
from tsai.export import get_nb_name
nb_name = get_nb_name()
create_scripts(nb_name);